In [61]:
from bs4 import BeautifulSoup
import requests
import datetime
import time
import mysql.connector
import re
from decimal import *

In [62]:
mydb = mysql.connector.connect(
   host="localhost",
#     host="127.0.0.1",
    port=3306,
    user="root",
    password="AskYoMama",
    database="options_screener"
)

mycursor = mydb.cursor()

mycursor.execute("USE options_screener")

In [3]:
d='December 17, 2020'
da = datetime.datetime.strptime(d, '%B %d, %Y')
da

datetime.datetime(2020, 12, 17, 0, 0)

In [4]:
now = datetime.datetime.now()

In [5]:
(da - now).days

7

In [6]:
def get_dates(ticker='SPY', after_days=100):
    data_url = "https://finance.yahoo.com/quote/" + ticker + "/options"
    data_html = requests.get(data_url).content
    content = BeautifulSoup(data_html, "html.parser")

    all_contract_dates = content.find_all("select")[0].find_all("option")
    contract_dates = []
    now = datetime.datetime.now()
    for contract_date in all_contract_dates:
        
        d = datetime.datetime.strptime(contract_date.text, '%B %d, %Y')
        if (d - now).days >= after_days:
            contract_dates.append({
                "value": contract_date["value"],
                "date": contract_date.text
            })

    return contract_dates

In [7]:
def fetch_options_tables(ticket='SPY',date='1607299200'):
    data_url = "https://finance.yahoo.com/quote/SPY/options?date="+date
    data_html = requests.get(data_url).content
    content = BeautifulSoup(data_html, "html.parser")
    options_tables = content.find_all("table")

    options_tables = []
    tables = content.find_all("table")
    for i in range(0, len(content.find_all("table"))):
        options_tables.append(tables[i])
        
    return options_tables

In [8]:
contract_dates = get_dates(ticker='SPY', after_days=100)
contract_dates

[{'value': '1617148800', 'date': 'March 31, 2021'},
 {'value': '1618531200', 'date': 'April 16, 2021'},
 {'value': '1621555200', 'date': 'May 21, 2021'},
 {'value': '1623974400', 'date': 'June 18, 2021'},
 {'value': '1625011200', 'date': 'June 30, 2021'},
 {'value': '1631836800', 'date': 'September 17, 2021'},
 {'value': '1632960000', 'date': 'September 30, 2021'},
 {'value': '1639699200', 'date': 'December 17, 2021'},
 {'value': '1642723200', 'date': 'January 21, 2022'},
 {'value': '1647561600', 'date': 'March 18, 2022'},
 {'value': '1655424000', 'date': 'June 17, 2022'},
 {'value': '1663286400', 'date': 'September 16, 2022'},
 {'value': '1671148800', 'date': 'December 16, 2022'},
 {'value': '1674172800', 'date': 'January 20, 2023'}]

In [9]:
options_tables = fetch_options_tables(ticket='SPY',date='1607299200')

In [42]:
Decimal('0.00')

Decimal('0.00')

In [85]:
def extract_info_from_table(options):
    options = options.find_all("tr")[1:]
    all_info = []
    for option in options:

        call_data = []
        for td in BeautifulSoup(str(option), "html.parser").find_all("td"):
            call_data.append(td.text)

        if call_data[8] == '-':
            call_data[8] = 0
            
        ticker = re.findall(r"^[A-Z]+", call_data[0])[0]
        contract = call_data[0]
        
        expiry = datetime.date(2000+int(contract[len(ticker):len(ticker)+2]),
                     int(contract[len(ticker)+2:len(ticker)+4]),
                     int(contract[len(ticker)+4:len(ticker)+6]))

        call_info = {
            'contract': call_data[0],
            'ticker': ticker,
            'type': re.findall(r"[A-Z]+", call_data[0][5:])[0],
            'strike': call_data[2],
            'last': call_data[3], 
            'bid': call_data[4],
            'ask': call_data[5],
            'volume': call_data[8],
            'iv': call_data[10][:-1],
            'expiry': expiry
        }

        all_info.append(call_info)
    return all_info

In [55]:
all_tables = []
for contract_date in contract_dates:
    # fetches from yahoo
    call_and_put_options_tables = fetch_options_tables(ticket='SPY',date=contract_date["value"])
    all_tables = all_tables + call_and_put_options_tables

In [86]:
for table in all_tables:
    extract_info_from_table(table)

In [35]:
extract_info_from_table(all_tables[1])

[{'contract': 'SPY210331P00190000',
  'ticker': ['SPY'],
  'type': ['P'],
  'strike': '190.00',
  'last': '0.30',
  'bid': '0.00',
  'ask': '0.00',
  'volume': '6',
  'iv': '25.00%'},
 {'contract': 'SPY210331P00195000',
  'ticker': ['SPY'],
  'type': ['P'],
  'strike': '195.00',
  'last': '0.36',
  'bid': '0.00',
  'ask': '0.00',
  'volume': '1',
  'iv': '25.00%'},
 {'contract': 'SPY210331P00200000',
  'ticker': ['SPY'],
  'type': ['P'],
  'strike': '200.00',
  'last': '0.37',
  'bid': '0.00',
  'ask': '0.00',
  'volume': '14',
  'iv': '25.00%'},
 {'contract': 'SPY210331P00205000',
  'ticker': ['SPY'],
  'type': ['P'],
  'strike': '205.00',
  'last': '0.40',
  'bid': '0.00',
  'ask': '0.00',
  'volume': '10',
  'iv': '25.00%'},
 {'contract': 'SPY210331P00210000',
  'ticker': ['SPY'],
  'type': ['P'],
  'strike': '210.00',
  'last': '0.50',
  'bid': '0.00',
  'ask': '0.00',
  'volume': '55',
  'iv': '25.00%'},
 {'contract': 'SPY210331P00215000',
  'ticker': ['SPY'],
  'type': ['P'],
  '

In [87]:
def add_table_to_db(table_as_json):
    for contract in table_as_json:
        print(contract)
        add_contract = """
            INSERT IGNORE INTO contracts
            (name, ticker, type, strike, last, bid, ask, vol, iv, expiry)
            VALUES
            (%(contract)s, %(ticker)s, %(type)s, %(strike)s, %(last)s, %(bid)s,
            %(ask)s, %(volume)s, %(iv)s,%(expiry)s )
        """
        mycursor.execute(add_contract, contract)
        mydb.commit()

In [88]:
for table in all_tables:
    table_as_json = extract_info_from_table(table)
    add_table_to_db(table_as_json)

{'contract': 'SPY210331C00200000', 'ticker': 'SPY', 'type': 'C', 'strike': '200.00', 'last': '169.98', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331C00205000', 'ticker': 'SPY', 'type': 'C', 'strike': '205.00', 'last': '133.80', 'bid': '123.53', 'ask': '124.69', 'volume': '50', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331C00210000', 'ticker': 'SPY', 'type': 'C', 'strike': '210.00', 'last': '124.78', 'bid': '136.64', 'ask': '137.80', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331C00220000', 'ticker': 'SPY', 'type': 'C', 'strike': '220.00', 'last': '149.20', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331C00222000', 'ticker': 'SPY', 'type': 'C', 'strike': '222.00', 'last': '79.77', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2021, 

{'contract': 'SPY210331C00460000', 'ticker': 'SPY', 'type': 'C', 'strike': '460.00', 'last': '0.12', 'bid': '0.00', 'ask': '0.00', 'volume': '10', 'iv': '6.25', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331C00465000', 'ticker': 'SPY', 'type': 'C', 'strike': '465.00', 'last': '0.10', 'bid': '0.00', 'ask': '0.00', 'volume': '3', 'iv': '6.25', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331C00470000', 'ticker': 'SPY', 'type': 'C', 'strike': '470.00', 'last': '0.09', 'bid': '0.00', 'ask': '0.00', 'volume': '15', 'iv': '6.25', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331P00190000', 'ticker': 'SPY', 'type': 'P', 'strike': '190.00', 'last': '0.30', 'bid': '0.00', 'ask': '0.00', 'volume': '6', 'iv': '25.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331P00195000', 'ticker': 'SPY', 'type': 'P', 'strike': '195.00', 'last': '0.36', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '25.00', 'expiry': datetime.date(2021, 3, 31)}
{'cont

{'contract': 'SPY210331P00376000', 'ticker': 'SPY', 'type': 'P', 'strike': '376.00', 'last': '19.67', 'bid': '0.00', 'ask': '0.00', 'volume': '10', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331P00377000', 'ticker': 'SPY', 'type': 'P', 'strike': '377.00', 'last': '20.18', 'bid': '0.00', 'ask': '0.00', 'volume': '10', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331P00378000', 'ticker': 'SPY', 'type': 'P', 'strike': '378.00', 'last': '48.60', 'bid': '54.64', 'ask': '56.18', 'volume': 0, 'iv': '60.56', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331P00379000', 'ticker': 'SPY', 'type': 'P', 'strike': '379.00', 'last': '26.40', 'bid': '0.00', 'ask': '0.00', 'volume': '9', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'contract': 'SPY210331P00380000', 'ticker': 'SPY', 'type': 'P', 'strike': '380.00', 'last': '23.00', 'bid': '0.00', 'ask': '0.00', 'volume': '4', 'iv': '0.00', 'expiry': datetime.date(2021, 3, 31)}
{'

{'contract': 'SPY210416P00165000', 'ticker': 'SPY', 'type': 'P', 'strike': '165.00', 'last': '0.25', 'bid': '0.00', 'ask': '0.00', 'volume': '3', 'iv': '25.00', 'expiry': datetime.date(2021, 4, 16)}
{'contract': 'SPY210416P00170000', 'ticker': 'SPY', 'type': 'P', 'strike': '170.00', 'last': '0.33', 'bid': '0.00', 'ask': '0.00', 'volume': '5', 'iv': '25.00', 'expiry': datetime.date(2021, 4, 16)}
{'contract': 'SPY210416P00175000', 'ticker': 'SPY', 'type': 'P', 'strike': '175.00', 'last': '0.29', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '25.00', 'expiry': datetime.date(2021, 4, 16)}
{'contract': 'SPY210416P00180000', 'ticker': 'SPY', 'type': 'P', 'strike': '180.00', 'last': '0.34', 'bid': '0.00', 'ask': '0.00', 'volume': '2', 'iv': '25.00', 'expiry': datetime.date(2021, 4, 16)}
{'contract': 'SPY210416P00185000', 'ticker': 'SPY', 'type': 'P', 'strike': '185.00', 'last': '0.38', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '25.00', 'expiry': datetime.date(2021, 4, 16)}
{'con

{'contract': 'SPY210521C00200000', 'ticker': 'SPY', 'type': 'C', 'strike': '200.00', 'last': '131.46', 'bid': '162.76', 'ask': '164.71', 'volume': 0, 'iv': '0.00', 'expiry': datetime.date(2021, 5, 21)}
{'contract': 'SPY210521C00220000', 'ticker': 'SPY', 'type': 'C', 'strike': '220.00', 'last': '149.63', 'bid': '0.00', 'ask': '0.00', 'volume': '47', 'iv': '0.00', 'expiry': datetime.date(2021, 5, 21)}
{'contract': 'SPY210521C00225000', 'ticker': 'SPY', 'type': 'C', 'strike': '225.00', 'last': '107.77', 'bid': '138.16', 'ask': '140.01', 'volume': 0, 'iv': '0.00', 'expiry': datetime.date(2021, 5, 21)}
{'contract': 'SPY210521C00235000', 'ticker': 'SPY', 'type': 'C', 'strike': '235.00', 'last': '112.08', 'bid': '133.91', 'ask': '135.00', 'volume': '10', 'iv': '46.98', 'expiry': datetime.date(2021, 5, 21)}
{'contract': 'SPY210521C00240000', 'ticker': 'SPY', 'type': 'C', 'strike': '240.00', 'last': '120.00', 'bid': '0.00', 'ask': '0.00', 'volume': '5', 'iv': '0.00', 'expiry': datetime.date(202

...

{'contract': 'SPY220916P00140000', 'ticker': 'SPY', 'type': 'P', 'strike': '140.00', 'last': '1.60', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '12.50', 'expiry': datetime.date(2022, 9, 16)}
{'contract': 'SPY220916P00150000', 'ticker': 'SPY', 'type': 'P', 'strike': '150.00', 'last': '2.18', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '12.50', 'expiry': datetime.date(2022, 9, 16)}
{'contract': 'SPY220916P00155000', 'ticker': 'SPY', 'type': 'P', 'strike': '155.00', 'last': '3.99', 'bid': '1.71', 'ask': '2.74', 'volume': '1', 'iv': '39.98', 'expiry': datetime.date(2022, 9, 16)}
{'contract': 'SPY220916P00160000', 'ticker': 'SPY', 'type': 'P', 'strike': '160.00', 'last': '2.69', 'bid': '0.00', 'ask': '0.00', 'volume': '20', 'iv': '12.50', 'expiry': datetime.date(2022, 9, 16)}
{'contract': 'SPY220916P00170000', 'ticker': 'SPY', 'type': 'P', 'strike': '170.00', 'last': '5.18', 'bid': '5.56', 'ask': '6.40', 'volume': 0, 'iv': '44.24', 'expiry': datetime.date(2022, 9, 16)}
{'cont

{'contract': 'SPY221216C00115000', 'ticker': 'SPY', 'type': 'C', 'strike': '115.00', 'last': '194.70', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216C00120000', 'ticker': 'SPY', 'type': 'C', 'strike': '120.00', 'last': '214.78', 'bid': '224.54', 'ask': '228.28', 'volume': '50', 'iv': '0.00', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216C00130000', 'ticker': 'SPY', 'type': 'C', 'strike': '130.00', 'last': '204.37', 'bid': '215.37', 'ask': '217.55', 'volume': '50', 'iv': '0.00', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216C00135000', 'ticker': 'SPY', 'type': 'C', 'strike': '135.00', 'last': '192.20', 'bid': '209.27', 'ask': '211.59', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216C00140000', 'ticker': 'SPY', 'type': 'C', 'strike': '140.00', 'last': '219.11', 'bid': '0.00', 'ask': '0.00', 'volume': '3', 'iv': '0.00', 'expiry': datetime.d

{'contract': 'SPY221216P00085000', 'ticker': 'SPY', 'type': 'P', 'strike': '85.00', 'last': '0.59', 'bid': '0.00', 'ask': '0.00', 'volume': '129', 'iv': '25.00', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216P00090000', 'ticker': 'SPY', 'type': 'P', 'strike': '90.00', 'last': '0.75', 'bid': '0.00', 'ask': '0.00', 'volume': '67', 'iv': '25.00', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216P00095000', 'ticker': 'SPY', 'type': 'P', 'strike': '95.00', 'last': '0.75', 'bid': '0.00', 'ask': '0.00', 'volume': '2', 'iv': '12.50', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216P00100000', 'ticker': 'SPY', 'type': 'P', 'strike': '100.00', 'last': '0.80', 'bid': '0.00', 'ask': '0.00', 'volume': '7', 'iv': '12.50', 'expiry': datetime.date(2022, 12, 16)}
{'contract': 'SPY221216P00105000', 'ticker': 'SPY', 'type': 'P', 'strike': '105.00', 'last': '1.03', 'bid': '0.00', 'ask': '0.00', 'volume': '3', 'iv': '12.50', 'expiry': datetime.date(2022, 12, 16)}


{'contract': 'SPY230120C00175000', 'ticker': 'SPY', 'type': 'C', 'strike': '175.00', 'last': '159.64', 'bid': '186.01', 'ask': '191.00', 'volume': 0, 'iv': '0.00', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120C00180000', 'ticker': 'SPY', 'type': 'C', 'strike': '180.00', 'last': '187.50', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '0.00', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120C00185000', 'ticker': 'SPY', 'type': 'C', 'strike': '185.00', 'last': '183.00', 'bid': '0.00', 'ask': '0.00', 'volume': '2', 'iv': '0.00', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120C00190000', 'ticker': 'SPY', 'type': 'C', 'strike': '190.00', 'last': '180.76', 'bid': '0.00', 'ask': '0.00', 'volume': '3', 'iv': '0.00', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120C00195000', 'ticker': 'SPY', 'type': 'C', 'strike': '195.00', 'last': '173.00', 'bid': '0.00', 'ask': '0.00', 'volume': '4', 'iv': '0.00', 'expiry': datetime.date(2023, 1, 20)

{'contract': 'SPY230120P00250000', 'ticker': 'SPY', 'type': 'P', 'strike': '250.00', 'last': '13.83', 'bid': '0.00', 'ask': '0.00', 'volume': '3', 'iv': '6.25', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120P00255000', 'ticker': 'SPY', 'type': 'P', 'strike': '255.00', 'last': '14.78', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '6.25', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120P00260000', 'ticker': 'SPY', 'type': 'P', 'strike': '260.00', 'last': '13.67', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '6.25', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120P00265000', 'ticker': 'SPY', 'type': 'P', 'strike': '265.00', 'last': '16.00', 'bid': '0.00', 'ask': '0.00', 'volume': '11', 'iv': '3.13', 'expiry': datetime.date(2023, 1, 20)}
{'contract': 'SPY230120P00270000', 'ticker': 'SPY', 'type': 'P', 'strike': '270.00', 'last': '17.15', 'bid': '0.00', 'ask': '0.00', 'volume': '1', 'iv': '3.13', 'expiry': datetime.date(2023, 1, 20)}
{'co